<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Легенда

Вы — аналитик данных в розничной компании, развивающей мобильное приложение. Команда продукта хочет понять:

* как пользователи взаимодействуют с приложением: сколько установок, сколько просматривают товары, сколько покупают;
* где теряются пользователи (на каком этапе воронки);
* какова эффективность разных каналов привлечения.

### Таблицы БД

**1. `app_sessions`** — установки приложения

| Поле                  | Тип SQL        | Описание                                        |
| --------------------- | -------------- | ----------------------------------------------- |
| `session_id`          | `VARCHAR(20)`  | Уникальный идентификатор установки              |
| `device_code`         | `VARCHAR(20)`  | Идентификатор устройства                        |
| `first_seen`          | `DATE`         | Дата установки                                  |
| `os_type`             | `VARCHAR(10)`  | Платформа (`iOS`, `Android`)                    |
| `acquisition_channel` | `VARCHAR(50)`  | Канал установки (`Organic`, `Facebook`, и т.д.) |
| `cpi_uah`             | `NUMERIC(6,2)` | Стоимость установки (Cost Per Install) в грн    |


> **`cpi_uah` (Cost Per Install)** - это сумма, которую компания платит рекламной платформе (например, Facebook, Google Ads) за то, что пользователь установил приложение по рекламе.
> * Для `Organic` установок значение обычно `0`
> * Используется для оценки эффективности каналов и расчёта окупаемости (ROI)

---

**2. `product_views_log`** — просмотры товаров

| Поле          | Тип SQL       | Описание                         |
| ------------- | ------------- | -------------------------------- |
| `device_code` | `VARCHAR(20)` | Устройство                       |
| `view_date`   | `DATE`        | Дата просмотра                   |
| `platform`    | `VARCHAR(10)` | Платформа                        |
| `view_count`  | `INTEGER`     | Количество просмотренных товаров |

---

**3. `devices_users_map`** — соответствие `device_code` и `user_uuid`

| Поле          | Тип SQL       | Описание                                |
| ------------- | ------------- | --------------------------------------- |
| `device_code` | `VARCHAR(20)` | Устройство                              |
| `user_uuid`   | `VARCHAR(20)` | Пользователь (присваивается при логине) |

> 🔎 Пользователь может не авторизоваться — тогда `user_uuid` будет отсутствовать.

---

**4. `orders_log`** — покупки

| Поле         | Тип SQL         | Описание                                   |
| ------------ | --------------- | ------------------------------------------ |
| `user_uuid`  | `VARCHAR(20)`   | Уникальный ID авторизованного пользователя |
| `order_time` | `DATE`          | Дата заказа                                |
| `total_uah`  | `NUMERIC(10,2)` | Сумма покупки в гривнах                    |

---

| Связь                                                      | Описание                          |
| ---------------------------------------------------------- | --------------------------------- |
| `app_sessions.device_code = devices_users_map.device_code` | Склейка установки с пользователем |
| `devices_users_map.user_uuid = orders_log.user_uuid`       | Кто сделал заказ                  |
| `product_views_log.device_code = app_sessions.device_code` | Кто смотрел товары                |

---

### Путь пользователя

```text
УСТАНОВИЛ → СМОТРЕЛ → АВТОРИЗОВАЛСЯ → КУПИЛ
(app_sessions) → (product_views_log) → (devices_users_map) → (orders_log)
```

## SQL - запросы на создание таблиц

```sql
DROP TABLE IF EXISTS orders_log;
DROP TABLE IF EXISTS devices_users_map;
DROP TABLE IF EXISTS product_views_log;
DROP TABLE IF EXISTS app_sessions;

CREATE TABLE app_sessions (
    session_id VARCHAR(10) PRIMARY KEY,
    device_code VARCHAR(20) UNIQUE,
    first_seen DATE,
    os_type VARCHAR(10),
    acquisition_channel VARCHAR(50),
    cpi_uah NUMERIC(6, 2)
);

CREATE TABLE product_views_log (
    device_code VARCHAR(20),
    view_date DATE,
    platform VARCHAR(10),
    view_count INTEGER,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE devices_users_map (
    device_code VARCHAR(20),
    user_uuid VARCHAR(20) UNIQUE,
    FOREIGN KEY (device_code) REFERENCES app_sessions(device_code)
);

CREATE TABLE orders_log (
    user_uuid VARCHAR(20),
    order_time DATE,
    total_uah NUMERIC(10, 2),
    FOREIGN KEY (user_uuid) REFERENCES devices_users_map(user_uuid)
);
```

## Подключение к БД

In [1]:
DB_USER = "da_test_user"
DB_PASS = "c89M2psBdXGtBXpY3BPrTRZ2bXl6yq2i"
DB_HOST = "dpg-d1hvnobe5dus739ipbcg-a.frankfurt-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "da_test"

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text

In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [5]:
from sqlalchemy import inspect
inspector = inspect(engine)
print(inspector.get_table_names())

['app_sessions', 'product_views_log', 'devices_users_map', 'orders_log']


In [6]:
columns = inspector.get_columns('app_sessions')
for column in columns:
    print(column["name"], column["type"])

session_id VARCHAR(10)
device_code VARCHAR(20)
first_seen DATE
os_type VARCHAR(10)
acquisition_channel VARCHAR(50)
cpi_uah NUMERIC(6, 2)


## SELECT, FROM, WHERE — основа SQL-запроса

Каждый SQL-запрос начинается с **четкой структуры**:

```sql
SELECT [что выбрать]
FROM [откуда взять]
WHERE [какие строки отфильтровать]
```

### 🔹 SELECT

* Указывает, **какие столбцы** мы хотим получить
* Можно указать `*`, чтобы взять все столбцы (на практике — только для отладки)
* Поддерживает **выражения** (арифметика, функции, переименование через `AS`)

> ❗ **Порядок строк не гарантируется!**
> Если вам важен порядок — используйте `ORDER BY`

### 🔹 FROM

* Обязательно указывает, **из какой таблицы** брать данные
* Может быть не только одна таблица (будет позже, с `JOIN`)

### 🔹 WHERE

* Фильтрация строк до всех остальных операций (до `GROUP BY`, `HAVING`)
* Работает только со строками, которые **есть** в таблице

> ❗ **WHERE не может использовать агрегатные функции** (например, `AVG()`)

---

In [7]:
query = 'SELECT * FROM app_sessions'
df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00001,DVC0001,2024-01-15,Android,Referral,13.91
2,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
3,S00003,DVC0003,2024-02-23,iOS,Referral,14.98
4,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64
...,...,...,...,...,...,...
495,S00495,DVC0495,2024-06-20,iOS,Organic,19.58
496,S00496,DVC0496,2024-02-08,Android,Google Ads,34.12
497,S00497,DVC0497,2024-01-10,Android,Referral,20.56
498,S00498,DVC0498,2024-02-11,Android,Organic,27.31


In [12]:
user_input = "2024-03-01' OR '1'='1"
query = f"""
SELECT * FROM app_sessions
WHERE first_seen >= '{user_input}'
"""

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00001,DVC0001,2024-01-15,Android,Referral,13.91
2,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
3,S00003,DVC0003,2024-02-23,iOS,Referral,14.98
4,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64
...,...,...,...,...,...,...
495,S00495,DVC0495,2024-06-20,iOS,Organic,19.58
496,S00496,DVC0496,2024-02-08,Android,Google Ads,34.12
497,S00497,DVC0497,2024-01-10,Android,Referral,20.56
498,S00498,DVC0498,2024-02-11,Android,Organic,27.31


⚠️ **Пример небезопасного кода (SQL-инъекция):**

Представим, что ты вставляешь значение напрямую в строку запроса (так делать нельзя!):

```python
user_input = "2024-03-01' OR '1'='1"
query = f"""
SELECT * FROM app_sessions
WHERE first_seen >= '{user_input}'
"""

print(query)  # Вывод запроса

df = pd.read_sql(query, con=engine)  # ⚠️ Опасно!
```

👉 Что получится:

```sql
SELECT * FROM app_sessions
WHERE first_seen >= '2024-03-01' OR '1'='1'
```

🔴 **Что делает эта "инъекция"?**
Условие `OR '1'='1'` всегда истинно → фильтр `first_seen >= ...` перестаёт работать.
**Результат: возвращаются все строки** — потенциальная утечка данных.

In [16]:
query = text(
    """
    SELECT * FROM app_sessions
    WHERE first_seen >= :user_input
    """
)

df = pd.read_sql(query, con=engine, params={"user_input": "2024-03-01"})
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64
2,S00005,DVC0005,2024-06-02,Android,Facebook,10.67
3,S00006,DVC0006,2024-03-02,iOS,Referral,30.56
4,S00007,DVC0007,2024-05-25,iOS,Organic,15.28
...,...,...,...,...,...,...
333,S00492,DVC0492,2024-04-21,Android,Organic,31.41
334,S00493,DVC0493,2024-03-11,Android,Facebook,30.46
335,S00494,DVC0494,2024-04-18,iOS,Facebook,16.96
336,S00495,DVC0495,2024-06-20,iOS,Organic,19.58


## Логические операторы AND, OR, IN, BETWEEN, LIKE

#### `AND`, `OR`

* Позволяют соединять условия
* `AND` — обе части должны быть True
* `OR` — хотя бы одна часть True

> 📌 **Используйте скобки!** Логика без скобок может вас подвести

In [18]:
query = text(
    """
    SELECT *
    FROM app_sessions
    WHERE os_type = :plaform AND acquisition_channel = :channel
    """
)

df = pd.read_sql(query, con=engine, params={"plaform": "Android", "channel": "Facebook"})
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
2,S00005,DVC0005,2024-06-02,Android,Facebook,10.67
3,S00012,DVC0012,2024-05-31,Android,Facebook,17.41
4,S00026,DVC0026,2024-05-08,Android,Facebook,38.07
...,...,...,...,...,...,...
73,S00465,DVC0465,2024-04-05,Android,Facebook,35.70
74,S00477,DVC0477,2024-06-13,Android,Facebook,31.89
75,S00486,DVC0486,2024-02-12,Android,Facebook,31.48
76,S00487,DVC0487,2024-06-17,Android,Facebook,28.93


#### `IN (...)`

* Упрощает множественные сравнения
* Альтернатива множеству `OR`

In [20]:
channels = ["Google", "Facebook", "TikTok"]

res = '(' + ', '.join(channels) + ')'
res

'(Google, Facebook, TikTok)'

In [22]:
from sqlalchemy import bindparam

channels = ["Google", "Facebook", "TikTok"]

query = text(
    """
    SELECT *
    FROM app_sessions
    WHERE acquisition_channel IN ('Google', 'Facebook', 'TikTok')
    """
)

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
2,S00005,DVC0005,2024-06-02,Android,Facebook,10.67
3,S00010,DVC0010,2024-03-11,iOS,Facebook,32.80
4,S00012,DVC0012,2024-05-31,Android,Facebook,17.41
...,...,...,...,...,...,...
129,S00486,DVC0486,2024-02-12,Android,Facebook,31.48
130,S00487,DVC0487,2024-06-17,Android,Facebook,28.93
131,S00490,DVC0490,2024-01-23,iOS,Facebook,11.79
132,S00493,DVC0493,2024-03-11,Android,Facebook,30.46


In [25]:
from sqlalchemy import bindparam

channels = ["Google", "Facebook", "TikTok"]

query = text(
    """
    SELECT *
    FROM app_sessions
    WHERE acquisition_channel IN :channels
    """
).bindparams(
    bindparam("channels", expanding=True)
)

df = pd.read_sql(query, con=engine, params={'channels': channels})
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
2,S00005,DVC0005,2024-06-02,Android,Facebook,10.67
3,S00010,DVC0010,2024-03-11,iOS,Facebook,32.80
4,S00012,DVC0012,2024-05-31,Android,Facebook,17.41
...,...,...,...,...,...,...
129,S00486,DVC0486,2024-02-12,Android,Facebook,31.48
130,S00487,DVC0487,2024-06-17,Android,Facebook,28.93
131,S00490,DVC0490,2024-01-23,iOS,Facebook,11.79
132,S00493,DVC0493,2024-03-11,Android,Facebook,30.46


#### `BETWEEN a AND b`

* Диапазон значений (включительно)
* Удобно для фильтрации по датам, числам

In [26]:
query = text(
    """
    SELECT *
    FROM app_sessions
    WHERE first_seen BETWEEN '2024-02-01' AND '2024-03-01'
    """
)

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00003,DVC0003,2024-02-23,iOS,Referral,14.98
1,S00018,DVC0018,2024-02-04,iOS,Google Ads,26.72
2,S00023,DVC0023,2024-02-17,iOS,Referral,23.05
3,S00025,DVC0025,2024-02-26,Android,Referral,35.62
4,S00032,DVC0032,2024-02-28,iOS,Google Ads,22.83
...,...,...,...,...,...,...
67,S00469,DVC0469,2024-02-01,Android,Google Ads,21.85
68,S00478,DVC0478,2024-02-25,iOS,Organic,17.35
69,S00486,DVC0486,2024-02-12,Android,Facebook,31.48
70,S00496,DVC0496,2024-02-08,Android,Google Ads,34.12


#### `LIKE`

* Шаблон поиска (например, `"Goo%"`)
* `%` — любое количество любых символов
* `_` — ровно один любой символ

##### `LIKE` — **чувствителен к регистру** в **PostgreSQL**. Относительно других СУБД - нужно проверять.

```sql
SELECT * FROM users WHERE name LIKE 'Alex';
```

**Найдет только 'Alex'**, но **не** 'alex', 'ALEX' и т.п.

---

Если хочешь **регистронезависимый поиск**, используй:

1. `ILIKE` (PostgreSQL only)

```sql
SELECT * FROM users WHERE name ILIKE 'alex%';
```

Найдет `Alex`, `alex`, `ALEX`, `AlEx` и т.д.

2. `LOWER()` или `UPPER()`:

```sql
SELECT * FROM users WHERE LOWER(name) LIKE LOWER('alex%');
```

##### **%** = любой набор символов (в том числе пустой)

| Условие        | Найдёт строки, в которых...                                              |
| -------------- | ------------------------------------------------------------------------ |
| `LIKE 'Alex%'` | ...начинается с `'Alex'` (например: `Alex`, `Alexander`, `Alex123`)      |
| `LIKE '%son'`  | ...заканчивается на `'son'` (например: `Jackson`, `Emerson`)             |
| `LIKE '%lex%'` | ...содержится `'lex'` в любом месте (например: `Alex`, `Flex`, `Reflex`) |
| `LIKE '%'`     | ...все строки (потому что «любое количество любых символов»)             |

```sql
SELECT * FROM products
WHERE name LIKE '%phone%';
```

Найдёт: `smartphone`, `Phone Case`, `Headphones`, `my_phone_123`.

---

### Замена на `ILIKE` (для нечувствительного к регистру поиска):

```sql
SELECT * FROM products
WHERE name ILIKE '%phone%';
```

##### Символ `_` (один любой символ):

| Условие       | Найдёт                             |
| ------------- | ---------------------------------- |
| `LIKE '_ex'`  | `Lex`, `Rex`, но **не** `Alex`     |
| `LIKE 'A__x'` | `Alex`, `Abbx`, но не `Ax`, `Alxx` |

---

In [28]:
query = text("""
    SELECT *
    FROM app_sessions
    WHERE acquisition_channel LIKE '%o%'
  """
)
df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
2,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64
3,S00005,DVC0005,2024-06-02,Android,Facebook,10.67
4,S00008,DVC0008,2024-01-05,iOS,Google Ads,16.56
...,...,...,...,...,...,...
252,S00490,DVC0490,2024-01-23,iOS,Facebook,11.79
253,S00493,DVC0493,2024-03-11,Android,Facebook,30.46
254,S00494,DVC0494,2024-04-18,iOS,Facebook,16.96
255,S00496,DVC0496,2024-02-08,Android,Google Ads,34.12


In [30]:
query = text("""
    SELECT DISTINCT device_code
    FROM devices_users_map
    WHERE user_uuid LIKE '%99'
  """
)
df = pd.read_sql(query, con=engine)
df

,device_code
0,DVC0075
1,DVC0272
2,DVC0312


## ORDER BY, LIMIT


### `ORDER BY`

* Сортирует результат по указанному столбцу
* По умолчанию: **по возрастанию (`ASC`)**
* Указать `DESC` — по убыванию

> ❗ Без `ORDER BY` никакой порядок НЕ ГАРАНТИРУЕТСЯ, даже если таблица физически так хранится

In [31]:
query = text(
    """
      SELECT *
      FROM app_sessions
      ORDER BY cpi_uah DESC
    """
)

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00108,DVC0108,2024-02-29,Android,Google Ads,40.00
1,S00186,DVC0186,2024-04-14,Android,Referral,39.95
2,S00445,DVC0445,2024-04-27,Android,Organic,39.89
3,S00499,DVC0499,2024-05-07,iOS,Google Ads,39.78
4,S00041,DVC0041,2024-06-27,iOS,Facebook,39.77
...,...,...,...,...,...,...
495,S00419,DVC0419,2024-05-11,iOS,Facebook,10.22
496,S00176,DVC0176,2024-04-02,iOS,Referral,10.14
497,S00442,DVC0442,2024-02-18,iOS,Referral,10.13
498,S00297,DVC0297,2024-06-23,Android,Organic,10.05


In [32]:
query = text(
    """
      SELECT *
      FROM app_sessions
      ORDER BY cpi_uah DESC, first_seen ASC
    """
)

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00108,DVC0108,2024-02-29,Android,Google Ads,40.00
1,S00186,DVC0186,2024-04-14,Android,Referral,39.95
2,S00445,DVC0445,2024-04-27,Android,Organic,39.89
3,S00499,DVC0499,2024-05-07,iOS,Google Ads,39.78
4,S00041,DVC0041,2024-06-27,iOS,Facebook,39.77
...,...,...,...,...,...,...
495,S00419,DVC0419,2024-05-11,iOS,Facebook,10.22
496,S00176,DVC0176,2024-04-02,iOS,Referral,10.14
497,S00442,DVC0442,2024-02-18,iOS,Referral,10.13
498,S00297,DVC0297,2024-06-23,Android,Organic,10.05


### `LIMIT`

* Ограничивает количество строк
* Часто используется с `ORDER BY`, чтобы получить "топ-N"

In [33]:
query = text(
    """
      SELECT *
      FROM app_sessions
      ORDER BY cpi_uah DESC
      LIMIT 5
    """
)

df = pd.read_sql(query, con=engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00108,DVC0108,2024-02-29,Android,Google Ads,40.00
1,S00186,DVC0186,2024-04-14,Android,Referral,39.95
2,S00445,DVC0445,2024-04-27,Android,Organic,39.89
3,S00499,DVC0499,2024-05-07,iOS,Google Ads,39.78
4,S00041,DVC0041,2024-06-27,iOS,Facebook,39.77


## IS NULL — работа с пропущенными значениями

* В SQL значение `NULL` — это **"ничего"**, "неизвестно"
* Сравнивать `= NULL` нельзя! Используется `IS NULL` и `IS NOT NULL`

> ❗ `NULL` не участвует в арифметике и логике — результат всегда `NULL`

| Операция        | `NULL` даёт...     | Вместо этого             |
| --------------- | ------------------ | ------------------------ |
| `= NULL`        | `NULL` (не `TRUE`) | `IS NULL`                |
| `NULL + 1`      | `NULL`             | `COALESCE(col, 0) + 1`   |
| `NULL AND TRUE` | `NULL`             | Будь осторожен в `WHERE` |

---

In [35]:
query = text(
    """
      SELECT *
      FROM orders_log

    """
  )

df = pd.read_sql(query, con=engine)
df

,user_uuid,order_time,total_uah
0,USR0060,2024-03-08,683.38
1,USR0060,2024-01-27,2222.03
2,USR0060,2024-03-16,2924.31
3,USR0060,2024-06-01,2914.72
4,USR0060,2024-05-14,2313.61
...,...,...,...
2554,USR0211,2024-03-12,2518.68
2555,USR0211,2024-01-06,788.55
2556,USR0211,2024-02-17,1236.19
2557,USR0211,2024-04-30,1809.79


## DISTINCT — убрать дубликаты

* Убирает повторяющиеся строки по указанным столбцам
* Можно использовать в подзапросах или для подсчета уникальных значений

> ❗ `DISTINCT` всегда влияет на **всю комбинацию столбцов**, а не по отдельности

Оператор `DISTINCT` в SQL **удаляет дубликаты** — но он работает на **всю строку результата**, а не на каждый столбец по отдельности.

**Пример:**

| order\_id | user\_id | order\_time |
| --------- | -------- | ----------- |
| 1         | 101      | 2024-06-01  |
| 2         | 102      | 2024-06-01  |
| 3         | 101      | 2024-06-02  |
| 4         | 101      | 2024-06-01  |

---

```sql
SELECT DISTINCT user_id, order_time FROM orders_log;
```

Это вернёт **уникальные сочетания** `(user_id, order_time)`.

---

| user\_id | order\_time |
| -------- | ----------- |
| 101      | 2024-06-01  |
| 102      | 2024-06-01  |
| 101      | 2024-06-02  |



## Задачи

**Task 1.** Найти все установки, сделанные с февраля 2024 года по каналам `'Organic'` или `'Referral'`

**Task 2.** Вывести топ‑5 самых дорогих установок по стоимости `cpi_uah`

**Task 3.** Найти уникальные платформы, на которые устанавливали приложение

**Task 4.** Показать все `device_code`, которые встречаются в просмотрах, где `view_count > 30`

**Task 5.** Вывести заказы между 1 февраля и 1 марта 2024 года

**Task 6.** Найти устройства, у которых нет привязки к `user_uuid`